# Split the basin example base model

## Notebook Setup

In [ ]:
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib as pl
from shapely.geometry import Polygon, LineString

import flopy
from flopy.discretization import StructuredGrid
from flopy.mf6.utils import Mf6Splitter

In [ ]:
# import all plot style information from defaults.py
from defaults import *

### Parallel settings

1. Set the number of models in the row and column directions. NOTE: Set nrow_blocks or ncol_blocks to 0 to use metis.
2. Set path_to_mf6 to the path of parallel MODFLOW 6 (`path\to\mf6`) if it is not in your `PATH` otherwise set to `None`.


In [ ]:
nrow_blocks, ncol_blocks = 8, 0
path_to_mf6 = pl.Path.home() / ".local/bin/mf6" 

In [ ]:
# set a few variables based on parallel setting defined above
use_metis, nproc, parallel_ws = set_parallel_data(nrow_blocks, ncol_blocks)

#### Load the base basin model

In [ ]:
base_ws = pl.Path("../examples/ex-basin/basin_base")

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(
    sim_name="basin", 
    sim_ws=base_ws,
)

In [ ]:
base_gwf = base_sim.get_model()

### Split the base model

In [ ]:
mfsplit = Mf6Splitter(base_sim)

#### Generate the splitting array

In [ ]:
if use_metis:
    split_array = mfsplit.optimize_splitting_mask(nparts=nproc)
else:
    split_array = simple_mapping(
        nrow_blocks, 
        ncol_blocks, 
        base_gwf.modelgrid,
    )

#### Plot the splitting array

In [ ]:
fig, ax = plt.subplots(figsize=(figwidth, figwidth / 2))
pmv = flopy.plot.PlotMapView(model=base_gwf, ax=ax)
pa = pmv.plot_array(split_array)
pmv.plot_inactive()
plt.colorbar(pa, shrink=0.5)
plt.show()

#### Split the model

In [ ]:
new_sim = mfsplit.split_model(split_array)

In [ ]:
shutil.rmtree(parallel_ws, ignore_errors=True)
new_sim.set_sim_path(parallel_ws)

### Plot the submodels

Reassemble the top of the model from the submodels

In [ ]:
model_names = list(new_sim.model_names)
top_dict = {}
for modelname in model_names:
    mnum = int(modelname.split("_")[-1])
    top = new_sim.get_model(modelname).dis.top.array
    top_dict[mnum] = top
top_split = mfsplit.reconstruct_array(top_dict)

In [ ]:
fig, ax = plt.subplots(figsize=(figwidth, figwidth / 2))
ax.set_xlim(0, Lx)
ax.set_ylim(0, Ly)
ax.set_aspect("equal")
pmv = flopy.plot.PlotMapView(
    model=base_gwf,
    ax=ax,
)
pa = pmv.plot_array(top_split, vmin=vmin, vmax=vmax)
pmv.plot_inactive()
plt.colorbar(pa, shrink=0.5)
plt.show()

### Write the new simulation and other needed files 

In [ ]:
new_sim.write_simulation(silent=True)

#### Write the node mapping to a JSON file

In [ ]:
mfsplit.save_node_mapping(parallel_ws / "mfsplit_node_mapping.json")

#### Write the PETSc rc file

In [ ]:
write_petscdb(parallel_ws)

### Run the model in parallel

In [ ]:
if local_simulation():
    if path_to_mf6 is not None:
        new_sim.exe_name = path_to_mf6
    new_sim.run_simulation(
        processors=min(os.cpu_count(), nproc),
        )